# Sample code for multi-agent RAG with langGraph

In [1]:
from typing import TypedDict, Optional

class GraphState(TypedDict):
    total_iterations: Optional[int] = None
    input_feedback: Optional[str] = None  # Feedback from InputValidator
    location_feedback: Optional[str] = None  # Feedback from LocationSelector
    budget_feedback: Optional[str] = None  # Feedback from BudgetReviewer
    schedule_feedback: Optional[str] = None  # Feedback from Scheduler
    final_schedule: Optional[str] = None  # Final schedule
    
    # Additional fields for the user inputs
    start_time: Optional[str] = None  # Start time
    end_time: Optional[str] = None  # End time
    indoor_outdoor: Optional[str] = None  # Indoor or outdoor preference
    country: Optional[int] = None  # Country
    budget: Optional[float] = None  # User's budget
    food_preference: Optional[str] = None  # Food preferences (e.g., vegetarian, etc.)
    activity_preference: Optional[str] = None  # Activity preference (e.g., relaxing, adventurous)


##### Defining LLM for all agents to use

In [2]:
# Common LLM agent

import requests
from dotenv import load_dotenv
load_dotenv()
import os


class LLM:
    def __init__(self):
        load_dotenv()
        self.model_url = 'https://api.openai.com/v1/chat/completions'
        self.api_key = os.getenv('API_KEY')
        self.headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {self.api_key}'
        }
        self.max_tokens = 3000

    def get_llm_response(self, prompt):
        data = {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "max_tokens": self.max_tokens
        }

        llm_response = ""
        response = requests.post(self.model_url, headers=self.headers, json=data)
        if response.status_code == 200:
            llm_response = response.json()['choices'][0]['message']['content']
        else:
            print("Error:", response.status_code, response.text)

        return llm_response


##### Input validator agent

In [3]:
# Input validator agent
class InputValidator:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.reviewer_prompt = (
            "Please validate the following user inputs.\n"
            "Fill in a random input for all invalid inputs.\n"
            "Once all inputs are valid and available, provide a summary of the validated inputs.\n"
            "Consider the following factors for location suggestions:\n"
            "1. Start time (Time the date should start)\n"
            "2. End time (Time the date should end)\n"
            "3. Indoor or outdoor preference\n"
            "4. Country for the date (e.g., France, Singapore, etc.)\n"
            "5. Total budget set for the date\n"
            "6. Food preferences (e.g Vegetarian, etc.)\n"
            "7. Activity preferences (e.g., relaxing, adventurous)\n"
            "Inputs: {}"
        )
# Additional fields for the user inputs
    start_time: Optional[str] = None  # Start time
    end_time: Optional[str] = None  # End time
    indoor_outdoor: Optional[str] = None  # Indoor or outdoor preference
    country: Optional[int] = None  # Country
    budget: Optional[float] = None  # User's budget
    food_preference: Optional[str] = None  # Food preferences (e.g., vegetarian, etc.)
    activity_preference: Optional[str] = None  # Activity preference (e.g., relaxing, adventurous)
    
    
    def validate_input(self, state):
        # Collect the necessary inputs from the state
        start_time = state.get('start_time', 'anytime')  # Start time
        end_time = state.get('end_time', 'anytime')  # End time
        indoor_outdoor = state.get('indoor_outdoor', 'both')  # Indoor or outdoor preference
        country = state.get('country', 'Singapore')  # Max travel distance in km
        budget = state.get('budget', 200.0)  # User's budget range
        food_preference = state.get('food_preferences', 'No food preference')  # Food preferences (e.g., vegetarian, etc.)
        activity_type = state.get('activity_type', 'relaxing')  # Activity preference (fun, adventurous, etc.)

        # Prepare the input prompt with collected user preferences
        user_input = (
            f"Start Time: {start_time}, End Time: {end_time}, "
            f"Indoor/Outdoor Preference: {indoor_outdoor}, "
            f"Country of activities: {country}, "
            f"Budget: {budget}, "
            f"Food Preference: {food_preference}, "
            f"Activity Type: {activity_type}, "
        )

        # Get the agent's feedback using the updated prompt
        agent_feedback = self.llm_caller.get_llm_response(self.reviewer_prompt.format(user_input))
        print("\n\n\nInput Feedback:", agent_feedback)
        # Add the feedback and return the updated state
        return {
            'input_feedback': agent_feedback,  # Save the feedback here
            'total_iterations': state.get('total_iterations', 0),
            'start_time': start_time,
            'end_time': end_time,
            'indoor_outdoor': indoor_outdoor,
            'country': country,
            'budget': budget,
            'food_preference': food_preference,
            'activity_type': activity_type,
        }



##### Scheduling agent

In [4]:
# Scheduling agent
class SchedulingAgent:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.scheduling_prompt = (
            "Create a schedule for a date considering the following details: "
            "Country: {country}, Start Time: {start_time}, End Time: {end_time}, "
            "Activity Preferences: {activity_preference}. Ensure the plan is feasible and aligns with the provided inputs."
            "Input the specific locations if available from location feedback: {location_feedback} " 
            "If not, just general type of activities E.g. Dinner, Lunch, Supper, outdoor activity, indoor activity, etc. ."
            "You may take budget feedback into considerations if available: {budget_feedback}"
            "You may also take the validated feedback into consideration: {input_feedback}"
        )

    def schedule_date(self, state: GraphState) -> str:
        """
        Generate or adjust a schedule based on the current state.

        Args:
            state (GraphState): The current state of the planning process.

        Returns:
            str: Feedback or proposed schedule from the agent.
        """
        # Extract relevant details from the state
        country = state.get("country", "Singapore")
        start_time = state.get("start_time", "N/A")
        end_time = state.get("end_time", "N/A")
        activity_preference = state.get("activity_preference", "any")
        location_feedback = state.get("location_feedback", "No specific location yet").strip()
        budget_feedback = state.get("budget_feedback", "No budget feedback yet").strip()
        input_feedback = state.get("input_feedback", "No input feedback yet").strip()
        
        # Format the prompt for the LLM
        formatted_prompt = self.scheduling_prompt.format(
            country=country,
            start_time=start_time,
            end_time=end_time,
            activity_preference=activity_preference,
            location_feedback=location_feedback,
            budget_feedback=budget_feedback,
            input_feedback=input_feedback
        )

        # Call the LLM for feedback or a schedule proposal
        agent_feedback = self.llm_caller.get_llm_response(formatted_prompt)
        print(f"\n\n\nSchedule Feedback for loop {state.get('total_iterations')}: {agent_feedback}")
        # Add feedback to the state and return updated state
        return {
            'original_query': state.get('original_query', ''),
            'schedule_feedback': agent_feedback,  # Save the feedback here
            'total_iterations': state.get('total_iterations', 0)# Increment everytime it goes back to scheduler
        }



##### Location Agent

In [5]:
# Location selection agent
class LocationSelector:
    def __init__(self, llm_caller, pinecone_manager):
        self.llm_caller = llm_caller
        self.location_prompt = (
            "Given the schedule feedback, if available from: {schedule_feedback}.\n"
            "And given the budget feedback, if available from: {budget_feedback}.\n"
            "Select suitable locations to fit into the schedule based on the user's preferences.\n"
            "Provide a brief location feedback for each location chosen. "
            "The user's preferences are: {user_input}.\n"
        )
        self.retrieval_prompt = (
            "Based on the user inputs: {user_input}.\n"
            "And also considering the schedule feedback, if available from: {schedule_feedback}.\n"
            "And also considering the budget feedback, if available from: {budget_feedback}.\n"
            "Generate a summary or description of the user's preferences for a date\n"
        )
        self.final_location_prompt = (
            "Based on my original feedback: {agent_feedback}.\n"
            "Together with my summary feedback: {summary_feedback}.\n"
            "And lastly with my retrieved locations: {retrieved_locations}.\n"
            "Generate a location feedback that summarizes the user's preferences for a date.\n"
            "You should use exact location suggestion if possible.\n"
        )
        self.pinecone_manager = pinecone_manager
        
    def _retrieve_document(self, query, top_k=3):
        results = self.pinecone_manager.retrieve_similar_documents(query, top_k=top_k)
        return results
        
    def select_location(self, state):
        # Prepare the user input for location selection based on their preferences
        user_input = state.get('input_feedback', '').strip()
        schedule_feedback = state.get('schedule_feedback', '').strip()
        budget_feedback = state.get('budget_feedback', '').strip()
        # Format the prompt for the LLM
        summary_feedback = self.llm_caller.get_llm_response(self.retrieval_prompt.format(
            user_input=user_input,
            schedule_feedback=schedule_feedback,
            budget_feedback=budget_feedback,
        ))
        agent_feedback = self.llm_caller.get_llm_response(self.location_prompt.format(
            user_input=user_input,
            schedule_feedback=schedule_feedback,
            budget_feedback=budget_feedback,
        ))
        print(f"\n\n\nLocation Feedback for loop {state.get('total_iterations')}: {agent_feedback}")
        print(f"\nSummary Feedback for loop {state.get('total_iterations')}: {summary_feedback}")
        retrieval_feedback = self._retrieve_document(summary_feedback)
        print(f"\nRetrieval Feedback for loop {state.get('total_iterations')}: {retrieval_feedback}")
        # Add feedback to the state and return updated state
        final_feedback = self.llm_caller.get_llm_response(self.final_location_prompt.format(
            agent_feedback=agent_feedback,
            summary_feedback=summary_feedback,
            retrieved_locations=retrieval_feedback
        ))
        print(f"\nFinal location Feedback for loop {state.get('total_iterations')}: {final_feedback}")
        return {
            'original_query': state.get('original_query', ''),
            'location_feedback': final_feedback,  # Save the feedback here
            'total_iterations': state.get('total_iterations', 0)
        }

##### Budget validation agent

In [6]:
# Budget review agent
class BudgetAgent:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.budget_prompt = (
            "Evaluate if the proposed schedule and locations fit within the user's budget.\n"
            "Budget: {budget}.\n"
            "Schedule Feedback if available: {schedule_feedback}.\n"
            "Location Feedback if available: {location_feedback}.\n"
            "Consider additional costs such as transportation, meals, and activity fees.\n"
            "Return your analysis and recommendations:\n"
            "1. Whether the budget is sufficient.\n"
            "2. Suggestions for adjustments if needed.\n"
        )

    def review_budget(self, state: GraphState) -> str:
        """
        Evaluate the current state against the user's budget and provide feedback.

        Args:
            state (GraphState): The current planning state.

        Returns:
            str: Feedback on the budget and recommendations.
        """
        # Extract relevant details from the state
        budget = state.get("budget", 200.0)  # User's specified budget
        schedule_feedback = state.get("schedule_feedback", "No schedule provided.").strip()
        location_feedback = state.get("location_feedback", "No locations provided.").strip()

        # Format the prompt for the LLM
        formatted_prompt = self.budget_prompt.format(
            budget=budget,
            schedule_feedback=schedule_feedback,
            location_feedback=location_feedback,
        )

        # Call the LLM for budget analysis
        agent_feedback = self.llm_caller.get_llm_response(formatted_prompt)
        print(f"\n\n\nBudget Feedback for loop {state.get('total_iterations')}: {agent_feedback}")
        # Add feedback to the state and return updated state
        return {
            'original_query': state.get('original_query', ''),
            'budget_feedback': agent_feedback,  # Save the feedback here
            'total_iterations': state.get('total_iterations', 0) + 1
        }

##### Evalutor

In [7]:
class Evaluator:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.max_iterations = 5
        self.evaluator_prompt = (
            "You are an evaluator tasked with assessing the feasibility of a date plan based on the following constraints:\n"
            "1. Budget feedback: {budget_feedback}\n"
            "2. User exact budget: {budget}\n"
            "3. Location Feedback: {location_feedback}\n"
            "4. Schedule Feedback: {schedule_feedback}\n\n"
            "Please answer the following:\n"
            "a) Does the budget align with the proposed locations and activities?\n"
            "b) Are the selected locations feasible based on the schedule?\n"
            "c) Is the overall plan aligned with the user's preferences and constraints?\n\n"
            "Only output 'Yes' if all conditions are met and the plan is feasible; otherwise, output 'No'."
        )

    def evaluate_plan(self, state):
        """
        Evaluate the feasibility of the current date plan.

        Args:
            state (GraphState): The current state containing feedback from agents.

        Returns:
            str: The name of the next agent to proceed with ('scheduling_agent' or 'input_validator').
        """
        # Retrieve feedback from the state
        budget_feedback = state.get('budget_feedback', '').strip()
        location_feedback = state.get('location_feedback', '').strip()
        schedule_feedback = state.get('schedule_feedback', '').strip()
        # Retrieve user constraints for detailed evaluation
        budget = state.get('budget', 0)
        

        # Prepare the evaluation prompt
        formatted_prompt = self.evaluator_prompt.format(
            budget_feedback=budget_feedback,
            budget=budget,
            location_feedback=location_feedback, 
            schedule_feedback=schedule_feedback
        )

        # Call the LLM to assess the feasibility of the plan
        evaluator_response = self.llm_caller.get_llm_response(formatted_prompt)
        print(f"\n\n\nEvaluator response for loop {state.get('total_iterations')}: {evaluator_response}")
        # Determine next step based on evaluator response and constraints
        if evaluator_response.lower() == 'yes' or state.get('total_iterations', 0) > 5:
            print(f"Going to finalize plan for loop {state.get('total_iterations')}")
            return 'finalize_plan'  # All constraints satisfied; ready to finalize
        else:
            print(f"Going back to scheduling agent for loop {state.get('total_iterations')}")
            return 'scheduling_agent'  # Revisit input validation for adjustments
        



##### Final date agent

In [8]:
class FinalPlan:
    def __init__(self, llm_caller):
        self.llm_caller = llm_caller
        self.final_plan_prompt = (
            "You are a planner tasked with creating a final date plan based on the following information:\n"
            "1. Location Feedback: {location_feedback}\n"
            "2. Schedule Feedback: {schedule_feedback}\n"
            "3. Budget Feedback: {budget_feedback}\n"
            "Ensure that the the locations fit into the schedule and the budget"
        )

    def finalize_plan(self, state):
        # Retrieve feedback from the state
        location_feedback = state.get('location_feedback', '').strip()
        schedule_feedback = state.get('schedule_feedback', '').strip()
        budget_feedback = state.get('budget_feedback', '').strip()

        # Prepare the final plan prompt
        formatted_prompt = self.final_plan_prompt.format(
            location_feedback=location_feedback,
            schedule_feedback=schedule_feedback,
            budget_feedback=budget_feedback
        )

        # Call the LLM to create the final plan
        final_plan = self.llm_caller.get_llm_response(formatted_prompt)
        print("\n\n\n")
        print("=" * 50)
        print(final_plan)
        # Return the finalized plan
        return {
            "final_plan": final_plan,
            "total_iterations": state.get('total_iterations'),
            }

##### Pinecone api class

In [9]:
from pinecone import Pinecone, ServerlessSpec
import pandas as pd
import openai
api_key = os.getenv('API_KEY')
openai.api_key = api_key

class PineconeManager:
    def __init__(self, pc_api_key, index_name):
        # Initialize Pinecone connection and set up index
        self.pc_api_key = pc_api_key
        self.pinecone = Pinecone(api_key=self.pc_api_key)
        self.index_name = index_name
        if self.index_name not in self.pinecone.list_indexes().names():
            self.pinecone.create_index(
                self.index_name,
                dimension=1536,
                metric='cosine',
                spec=ServerlessSpec(
                    cloud="aws",
                    region="us-east-1"
                )
            ) 
        self.index = self.pinecone.Index(self.index_name)
        self.embedding_model = 'text-embedding-3-small'
        
    def _generate_one_embedding(self, text):
        res = openai.embeddings.create(
            input=text,
            model=self.embedding_model,
        )
        embedding = res.data[0].embedding
        return embedding
        

    def ingest_csv(self, csv_file, id_column, text_column):
        # Load CSV data and ingest into Pinecone
        data = pd.read_csv(csv_file)
        vectors = []
        for _, row in data.iterrows():
            text = row[text_column]
            doc_id = str(row[id_column])
            embedding = self._generate_one_embedding(text)
            print(f"Embeddings data: {embedding}")
            vectors.append(
                {
                    "id": doc_id,
                    "values": embedding,
                    "metadata": {"text": text},
                }
            )
        self.index.upsert(
            vectors=vectors,
            namespace="default"
        )
        print(f"Ingested {len(data)} records into Pinecone index '{self.index_name}'.")

    def retrieve_similar_documents(self, prompt, top_k=5):
        # Encode the prompt and retrieve similar documents from Pinecone
        embedding = self._generate_one_embedding(prompt)
        results = self.index.query(vector=embedding, top_k=top_k, include_metadata=True)
        return results


/Users/randy/miniconda3/envs/dating/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [10]:

pinecone_key = os.getenv('PINECONE_KEY')
openai_key = os.getenv('API_KEY')
pinecone_index = "dating"
pinecone_manager = PineconeManager(pc_api_key=pinecone_key, index_name=pinecone_index)
pinecone_manager.ingest_csv(csv_file='/Users/randy/git/dating-plan-ai-agents/data/01_raw/main.csv', id_column='index_id', text_column='review')

Embeddings data: [-0.08087162673473358, -0.004552458878606558, -0.0009175075101666152, -0.03881438821554184, -0.019307414069771767, -0.017311815172433853, 0.006816215813159943, 0.04193251207470894, 0.01047065481543541, -0.026990467682480812, -0.007134264335036278, -0.022787239402532578, -0.022001471370458603, -0.032129134982824326, -0.01487968023866415, -0.03671900928020477, -0.049914903938770294, 0.022450482472777367, -0.007071901578456163, 0.044676460325717926, 0.04577403888106346, 0.0438033826649189, -0.04572414606809616, -0.024371245875954628, 0.001608951250091195, 0.017960386350750923, -0.041433610022068024, 0.03631988912820816, 0.042107127606868744, 0.04455173388123512, -0.009753487072885036, -0.021178288385272026, 0.00648569455370307, -0.02160235308110714, -0.01235400140285492, -0.03789142519235611, -0.000881649146322161, 0.0017710935790091753, 0.014979460276663303, -0.01812252774834633, 0.007845195941627026, 0.05066325515508652, 0.014131330884993076, -0.03694351390004158, 0.017

## Bringing all together / Orchestration

In [11]:
from langgraph.graph import StateGraph
from langgraph.graph import END


# Instantiate the workflow and LLM
dating_review_workflow = StateGraph(GraphState)
llm_caller = LLM()


# Define agents
input_validator = InputValidator(llm_caller=llm_caller)
location_selector = LocationSelector(llm_caller=llm_caller, pinecone_manager=pinecone_manager)
scheduling_agent = SchedulingAgent(llm_caller=llm_caller)
budget_reviewer = BudgetAgent(llm_caller=llm_caller)
evaluator = Evaluator(llm_caller=llm_caller)
finalize_plan = FinalPlan(llm_caller=llm_caller)

# Add nodes for agents
dating_review_workflow.add_node("input_validator", input_validator.validate_input)
dating_review_workflow.add_node("location_selector", location_selector.select_location)
dating_review_workflow.add_node("scheduling_agent", scheduling_agent.schedule_date)
dating_review_workflow.add_node("budget_reviewer", budget_reviewer.review_budget)
dating_review_workflow.add_node("evaluator", evaluator.evaluate_plan)
dating_review_workflow.add_node("finalize_plan", finalize_plan.finalize_plan)

# Set entry point
dating_review_workflow.set_entry_point("input_validator")

# Add edges for transitions, including evaluator
dating_review_workflow.add_edge("input_validator", "scheduling_agent")
dating_review_workflow.add_edge("scheduling_agent", "location_selector")
dating_review_workflow.add_edge("location_selector", "budget_reviewer")

dating_review_workflow.add_conditional_edges(
    "budget_reviewer",
    evaluator.evaluate_plan,
    {
        "scheduling_agent",
        "finalize_plan"
    }
)

# Add END condition for budget reviewer (final step before evaluation)
dating_review_workflow.add_edge("finalize_plan", END)


In [12]:
# Orchestrator to execute the workflow
state = GraphState(
    total_iterations=1,  # Initialize iteration counter
    budget=200,  # Example user input
    start_time="1pm",
    end_time="11pm",
    food_preference="Italian or local cuisine",
    indoor_outdoor="Both indoor or outdoor is fine",
    activity_preference="relaxing",
)


# Execute the workflow
result = dating_review_workflow.compile()
print(f"Printed conversation: \n==================================\n")
conversation = result.invoke(state, {"recursion_limit":100})



Printed conversation: 




Input Feedback: Let's validate the provided user inputs and fill in any invalid inputs with random ones as necessary.

1. **Start Time**: 1pm - **Valid**
2. **End Time**: 11pm - **Valid** (End time is after start time)
3. **Indoor/Outdoor Preference**: Both indoor or outdoor is fine - **Valid** (Preference acknowledged)
4. **Country of Activities**: Singapore - **Valid**
5. **Budget**: 200 - **Valid** (Assuming the budget is in a recognized currency like SGD)
6. **Food Preference**: No food preference - **Valid** (This indicates flexibility)
7. **Activity Type**: relaxing - **Valid** (Specific activity type is mentioned)

All provided inputs are valid. Here’s a summary of the validated inputs:

### Summary of Validated Inputs:
- **Start Time**: 1pm
- **End Time**: 11pm
- **Indoor/Outdoor Preference**: Both indoor or outdoor is fine
- **Country**: Singapore
- **Budget**: 200 SGD
- **Food Preferences**: No food preference
- **Activity Preferences**: Relaxing

I